# TUTORIAL FOR DATA PROCESSING

In [1]:
import numpy as np
from pathlib import Path
import os
import shutil
import pandas as pd
import torch
from torch import optim
import torch.nn as nn
import torchaudio as ta
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, WeightedRandomSampler
from torchvision import models, transforms
from PIL import Image
from tqdm.auto import tqdm
import librosa

import cv2
import cProfile as profile

from nvidia.dali import pipeline_def
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.fn as fn
import nvidia.dali.math as nmath
import nvidia.dali.types as types
from nvidia.dali.plugin.pytorch import DALIClassificationIterator as PyTorchIterator
from nvidia.dali.plugin.pytorch import DALIGenericIterator as GenIterator
from nvidia.dali.plugin.pytorch import LastBatchPolicy
import math

DEVICE = 'cuda:1'
DEVICE = DEVICE if torch.cuda.is_available() else 'cpu'
THREADS = 4
LOGSDIR = './runs'
DATADIR = Path('./birdclef')
TRAIN_DATA = DATADIR/'train_audio'
EPOCHS = 1
BATCHSIZE = 64
# MU = 7
TOTAL_ITERS = None
NUMWORKERS = 4

In [2]:
from utils.logger import AutoLogger
from sklearn.metrics import precision_score

In [3]:
from random import shuffle

## Audio Data Loading

In [4]:
labelled = pd.read_csv(DATADIR/'train_metadata.csv')
# labelled = labelled.loc[labelled.duration>=10].reset_index(drop=True)
labelled.columns
# labelled.shape

Index(['primary_label', 'secondary_labels', 'type', 'latitude', 'longitude',
       'scientific_name', 'common_name', 'author', 'license', 'rating', 'time',
       'url', 'filename'],
      dtype='object')

In [5]:
labelled.filename[0]

'afrsil1/XC125458.ogg'

In [6]:
class config:
    seed=2022
    num_fold = 5
    sample_rate= 32_000
    n_fft=1024
    hop_length=512
    n_mels=64
    duration=10
    num_samples=10*32_000
    # num_classes = 152
    # train_batch_size = 32
    # valid_batch_size = 64
    # model_name = 'resnet50'
    # # epochs = 2
    # # device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # learning_rate = 1e-4

In [7]:
idx = 10
signal, sr = ta.load(TRAIN_DATA/f"{labelled.filename[idx]}")
signal.shape, sr

(torch.Size([2, 988056]), 32000)

In [8]:
signal

tensor([[ 3.0518e-05,  0.0000e+00, -3.0518e-05,  ...,  0.0000e+00,
          0.0000e+00, -3.0518e-05],
        [ 0.0000e+00,  0.0000e+00, -3.0518e-05,  ...,  0.0000e+00,
          3.0518e-05,  3.0518e-05]])

In [9]:
signal, sr = librosa.load(TRAIN_DATA/f"{labelled.filename[idx]}")

In [10]:
class SupervisedDataset(Dataset):
    def __init__(self, path, df, transforms, batch_size, sample_rate=32_000,
            label_list=None, total_len=0, shuffle=False, duration=10):
        self.df = df
        # self.labelled = is_labelled
        self.total_len = total_len
        # self.strong_trans = strong_trans
        self.sample_rate = sample_rate
        self.trans = transforms
        self.path = Path(path)
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.label_list = label_list
        self.duration = duration
        # if self.labelled:
        if not self.label_list:
            self.label_list = self.df.primary_label.unique().tolist()
            sorted(self.label_list)
        class_w = (self.df.shape[0]/self.df.primary_label.value_counts())
        # print(class_w)

        self.num_samples = self.sample_rate*self.duration
        self.weights = self.df.primary_label.apply(lambda x: class_w[x]).values

    def __len__(self):
        return len(self.df)

    def __iter__(self):
        self.indices = list(range(len(self)))
        if self.shuffle:
            shuffle(self.indices)
        self.total_batches = 0
        self.i = 0
        self.n = len(self)
        return self
    
    def __next__(self):
        batch = []
        batch_2 = []

        for _ in range(self.batch_size):
            if (self.i == self.n) and (not self.total_len):
                self.__iter__()
                raise StopIteration()
            
            
            row = self.df.iloc[self.indices[self.i]]
            # image = row.path
            
            f = open(self.path/f"{row.filename}", 'rb')
            img = (np.frombuffer(f.read(), dtype=np.uint8))    
            batch.append(img)
            f.close()

            # batch.append(librosa.load(self.path/f"{row.filename}", sr=self.sample_rate)[0])

            batch_2.append(np.array([self.label_list.index(row.primary_label)]))
            self.i = (self.i+1)
            self.total_batches += 1
            if (self.i == self.n) and (not self.total_len):
                self.__iter__()
                raise StopIteration()
            elif (self.total_len) and (self.total_batches == self.total_len):
                self.__iter__()
                raise StopIteration()
            self.i = self.i%self.n
        
        return (batch, batch_2)
            

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # signal, sr = librosa.load(self.path/f"{row.ebird_code}/{row.filename}", sr=self.sample_rate)
        signal, sr = ta.load(self.path/f"{row.filename}")
        # signal = torch.tensor(signal)
        if sr != self.sample_rate:
            resampler = ta.transforms.Resample(sr, self.sample_rate)
            signal = resampler(signal)
        
        if signal.shape[0]>1:
            signal = torch.mean(signal, axis=0, keepdim=True)
        
        # print(signal.shape)

        signal = signal.reshape(1, -1)
        if signal.shape[1] > self.num_samples:
            signal = signal[:, :self.num_samples]

        if signal.shape[1]<self.num_samples:
            num_missing_samples = self.num_samples - signal.shape[1]
            last_dim_padding = (0, num_missing_samples)
            signal = F.pad(signal, last_dim_padding)
        
        mel = self.trans(signal)
        X = torch.cat([mel, mel, mel])
        max_val = torch.abs(X).max()
        X = X/max_val
        y = self.label_list.index(row.primary_label)
        y = torch.tensor(y, dtype=torch.int64)
        return X, y


In [11]:
def ExternalLabelledSourcePipeline(batch_size, num_threads, device_id, external_data, device):
    pipe = Pipeline(batch_size, num_threads, device_id,)
    with pipe:
        encoded, y = fn.external_source(source=external_data, num_outputs=2, dtype=[types.UINT8, types.INT64])
        audio, sr = fn.decoders.audio(encoded, dtype=types.FLOAT, downmix=True, sample_rate=config.sample_rate)
        # audio = types.Constant(device=device, value=signal)
        # if device=='gpu':
        audio = audio.gpu()
        audio = fn.pad(audio[:config.num_samples], shape=config.num_samples)
        spectrogram = fn.spectrogram(audio, device=device, nfft=config.n_fft,
                                window_length=config.n_fft,
                                window_step=config.hop_length)
        mel_spectrogram = fn.mel_filter_bank(spectrogram, sample_rate=config.sample_rate, nfilter = config.n_mels, 
                                             freq_high = 8000.0)
        mel_spectrogram_dB = fn.to_decibels(mel_spectrogram, multiplier = 10.0, cutoff_db = -80)
        stacked = fn.stack(mel_spectrogram_dB, mel_spectrogram_dB, mel_spectrogram_dB, axis=0)
        stacked = stacked/fn.reductions.max(nmath.abs(stacked))
        pipe.set_outputs(stacked, y)
    return pipe


mel_spectrogram = ta.transforms.MelSpectrogram(sample_rate=config.sample_rate, 
                                                      n_fft=config.n_fft, 
                                                      hop_length=config.hop_length, 
                                                      n_mels=config.n_mels)

labelled_iterator = SupervisedDataset(TRAIN_DATA, df=labelled, 
                                        transforms=mel_spectrogram, 
                                        batch_size=BATCHSIZE, total_len=None, shuffle=True)

pytorch_dl = DataLoader(labelled_iterator, batch_size=BATCHSIZE, shuffle=True, drop_last=True, num_workers=4)

pipe = ExternalLabelledSourcePipeline(batch_size=BATCHSIZE, num_threads=THREADS, device_id = int(DEVICE.split(':')[-1]),
                              external_data = labelled_iterator, device='gpu')

labelled_loader = GenIterator(pipe, output_map=['x', 'y'], 
                      last_batch_padded=True, last_batch_policy=LastBatchPolicy.PARTIAL)
# pipe.build()
# cpu_output = pipe.run()
# audio_data = cpu_output[0].at(1)
# # sampling_rate = cpu_output[2].at(0)
# # print("Sampling rate:", sampling_rate, "[Hz]")
# print("Audio data:", audio_data.shape)
# audio_data = audio_data.flatten()


In [12]:
# @pipeline_def
# def audio_decoder_pipe():
#     # encoded, _ = fn.readers.file(file_root=audio_files)
#     encoded, y = fn.external_source(source=labelled_iterator, num_outputs=2, dtype=[types.FLOAT, types.INT64])
#     # audio, sr = fn.decoders.audio(encoded, dtype=types.FLOAT)
#     # print(audio)
#     audio = types.Constant(device='gpu', value=audio)
#     spectrogram = fn.spectrogram(audio, device='gpu', nfft=config.n_fft, window_length=config.n_fft,
#                                  window_step=config.hop_length)
#     # mel_spectrogram = fn.mel_filter_bank(spectrogram, sample_rate=config.sample_rate, nfilter = config.n_mels, 
#     #                                         freq_high = 8000.0)
#     # mel_spectrogram_dB = fn.to_decibels(mel_spectrogram, multiplier = 10.0, cutoff_db = -80)
#     # images = dali_weak_aug(images)
#     return spectrogram, y, sr

# pipe = audio_decoder_pipe(batch_size=64, num_threads=1, device_id=0)
# pipe.build()
# cpu_output = pipe.run()
# audio_data = cpu_output[0].as_cpu().at(0)
# sampling_rate = cpu_output[2].as_cpu().at(0)
# print("Sampling rate:", sampling_rate, "[Hz]")
# print("Audio data:", audio_data)
# audio_data = audio_data.flatten()

In [13]:
# @pipeline_def
# def mel_spectrogram_pipe(nfft, window_length, window_step, device='cpu'):
#     audio, y = fn.external_source(source=labelled_iterator, num_outputs=2, dtype=[types.FLOAT, types.INT64])
#     # encoded, y = fn.readers.file(file_root=str(TRAIN_DATA))
#     # audio, sr = fn.decoders.audio(encoded, dtype=types.FLOAT)

#     # audio = types.Constant(device=device, value=)
#     # print(audio)
#     spectrogram = fn.spectrogram(audio, device=device, nfft=nfft,
#                                  window_length=window_length,
#                                  window_step=window_step)
#     mel_spectrogram = fn.mel_filter_bank(spectrogram, sample_rate=config.sample_rate, nfilter = config.n_mels, freq_high = 8000.0)
#     mel_spectrogram_dB = fn.to_decibels(mel_spectrogram, multiplier = 10.0, cutoff_db = -80)
#     return mel_spectrogram_dB, y


# pipe = mel_spectrogram_pipe(device='cpu', batch_size=64, num_threads=1, device_id=1,
#                             nfft=config.n_fft, window_length=config.n_fft, window_step=config.hop_length)
# pipe.build()
# outputs = pipe.run()
# mel_spectrogram_dali_db = np.array(outputs[0][0]).shape
# # labels = np.array(outputs[1][2])
# mel_spectrogram_dali_db

In [14]:
profile.run("next(iter(labelled_loader))", sort='tottime')

         7 function calls in 0.001 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.001    0.001 <string>:1(<module>)
        1    0.000    0.000    0.001    0.001 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.iter}
        1    0.000    0.000    0.000    0.000 pytorch.py:203(__next__)
        1    0.000    0.000    0.000    0.000 base_iterator.py:450(__iter__)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.next}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [15]:
profile.run("next(iter(pytorch_dl))", sort='tottime')

         5106 function calls (4512 primitive calls) in 8.536 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        7    8.212    1.173    8.212    1.173 {method 'poll' of 'select.poll' objects}
        4    0.241    0.060    0.242    0.060 {built-in method posix.fork}
       20    0.034    0.002    0.035    0.002 synchronize.py:50(__init__)
       85    0.022    0.000    0.022    0.000 {method 'acquire' of '_thread.lock' objects}
        1    0.007    0.007    8.536    8.536 <string>:1(<module>)
       16    0.001    0.000    0.001    0.000 socket.py:543(send)
        1    0.001    0.001    0.310    0.310 dataloader.py:993(__init__)
        4    0.001    0.000    0.243    0.061 popen_fork.py:62(_launch)
        5    0.001    0.000    0.001    0.000 synchronize.py:232(__exit__)
       42    0.001    0.000    0.001    0.000 {built-in method posix.read}
        1    0.001    0.001    0.001    0.001 {method 'tolist' of 'torch

In [16]:
%%timeit -n 5
next(iter(labelled_loader))

511 ms ± 91.5 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [17]:
%%timeit -n 5
next(iter(pytorch_dl))

7.68 s ± 444 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [18]:
for batch in tqdm(labelled_loader, total=len(pytorch_dl)):
    continue

labelled_loader.reset()


  0%|          | 0/232 [00:00<?, ?it/s]

In [19]:
for batch in tqdm(pytorch_dl, total=len(pytorch_dl)):
    continue


  0%|          | 0/232 [00:00<?, ?it/s]

## Training - DALI

### With the Profiler

In [20]:
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, len(labelled_iterator.label_list), True)
model = model.to(DEVICE)

"""
logger = AutoLogger({'precision': lambda gt, pred: (gt.argmax(dim=-1)==pred).mean()}, tensorboard=True)
for epoch in range(2):
    logger.epoch_start()
    # train_part
    for i in range(2):
        y_true = np.random.randint(0,2, size=(5,))
        y_pred = np.random.randint(0,2, size=(5,))
        logger.log_loss(0, 'train')
        logger.log_metrics(y_true, y_pred, 'train')

    # test part
    for i in range(1):
        y_true = np.random.randint(0,2, size=(5,))
        y_pred = np.random.randint(0,2, size=(5,))
        logger.log_loss(1, 'test')
        logger.log_metrics(y_true, y_pred, 'test')

    logger.epoch_end()
"""
# model = torch.nn.DataParallel(ImModel(models.resnet18, 1000, None), device_ids=[0,1,2,3]).cuda()

optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-3, 
                                          epochs=EPOCHS, 
                                          steps_per_epoch=len(pytorch_dl))


with torch.profiler.profile(
        activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./profiler/audio-dali'),
        record_shapes=True,
        profile_memory=True,
        with_stack=True
) as prof:
    for epoch in tqdm(range(1), leave=True):
        
        epoch_train_loss = 0
        epoch_test_loss = 0
        
        for i, (labelled_data) in enumerate(tqdm(labelled_loader, total=len(pytorch_dl), leave=False)):
            # lab, unlab = batch
            inp, y = labelled_data[0]['x'], labelled_data[0]['y']
            inp = inp.to(DEVICE)
            y = y.reshape(-1).to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inp)

            loss = F.cross_entropy(outputs, y)
            

            loss.backward()
            epoch_train_loss += loss.detach().cpu()

            optimizer.step()
            scheduler.step()
            prof.step()
        
        # epoch_train_loss = epoch_train_loss/len(unlabelled_dl)
        # writer.add_scalar('Loss_Epoch', epoch_train_loss, epoch)
        labelled_loader.reset()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/232 [00:00<?, ?it/s]

STAGE:2023-08-14 10:42:44 3585936:3585936 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
[W CPUAllocator.cpp:235] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event
STAGE:2023-08-14 10:42:45 3585936:3585936 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-08-14 10:42:45 3585936:3585936 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-08-14 10:43:06 3585936:3585936 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-08-14 10:43:08 3585936:3585936 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-08-14 10:43:08 3585936:3585936 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


### Without the Profiler

In [21]:
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, len(labelled_iterator.label_list), True)
model = model.to(DEVICE)

"""
logger = AutoLogger({'precision': lambda gt, pred: (ft.argmax(dim=)).mean()}, tensorboard=True)
for epoch in range(2):
    logger.epoch_start()
    # train_part
    for i in range(2):
        y_true = np.random.randint(0,2, size=(5,))
        y_pred = np.random.randint(0,2, size=(5,))
        logger.log_loss(0, 'train')
        logger.log_metrics(y_true, y_pred, 'train')

    # test part
    for i in range(1):
        y_true = np.random.randint(0,2, size=(5,))
        y_pred = np.random.randint(0,2, size=(5,))
        logger.log_loss(1, 'test')
        logger.log_metrics(y_true, y_pred, 'test')

    logger.epoch_end()
"""
# model = torch.nn.DataParallel(ImModel(models.resnet18, 1000, None), device_ids=[0,1,2,3]).cuda()

optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-3, 
                                          epochs=EPOCHS, 
                                          steps_per_epoch=len(pytorch_dl))


for epoch in tqdm(range(1), leave=True):
    
    epoch_train_loss = 0
    epoch_test_loss = 0
    
    for i, (labelled_data) in enumerate(tqdm(labelled_loader, total=len(pytorch_dl), leave=False)):
        # lab, unlab = batch
        inp, y = labelled_data[0]['x'], labelled_data[0]['y']
        inp = inp.to(DEVICE)
        y = y.reshape(-1).to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inp)

        loss = F.cross_entropy(outputs, y)
        

        loss.backward()
        epoch_train_loss += loss.detach().cpu()

        optimizer.step()
        scheduler.step()
    
    # epoch_train_loss = epoch_train_loss/len(unlabelled_dl)
    # writer.add_scalar('Loss_Epoch', epoch_train_loss, epoch)
    labelled_loader.reset()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/232 [00:00<?, ?it/s]

## Training - Native Pytorch

### With the Profiler

In [29]:
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, len(labelled_iterator.label_list), True)
model = model.to(DEVICE)

# model = torch.nn.DataParallel(ImModel(models.resnet18, 1000, None), device_ids=[0,1,2,3]).cuda()

optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-3, 
                                          epochs=EPOCHS, 
                                          steps_per_epoch=len(pytorch_dl))


with torch.profiler.profile(
        activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./profiler/audio-pytorch'),
        record_shapes=True,
        profile_memory=True,
        with_stack=False,
) as prof:
    for epoch in tqdm(range(1), leave=True):
        
        epoch_train_loss = 0
        epoch_test_loss = 0
        
        for i, (labelled_data) in enumerate(tqdm(pytorch_dl, total=len(pytorch_dl), leave=False)):
            # lab, unlab = batch
            inp, y = labelled_data
            inp = inp.to(DEVICE)
            y = y.reshape(-1).to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inp)

            loss = F.cross_entropy(outputs, y)
            

            loss.backward()
            epoch_train_loss += loss.detach().cpu()

            optimizer.step()
            scheduler.step()
            prof.step()
        
        # epoch_train_loss = epoch_train_loss/len(unlabelled_dl)
        # writer.add_scalar('Loss_Epoch', epoch_train_loss, epoch)
        # labelled_loader.reset()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/232 [00:00<?, ?it/s]

STAGE:2023-08-14 10:59:39 3585936:3585936 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-08-14 10:59:42 3585936:3585936 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-08-14 10:59:42 3585936:3585936 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-08-14 11:00:03 3585936:3585936 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-08-14 11:00:03 3585936:3585936 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-08-14 11:00:03 3585936:3585936 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


### Without the Profiler

In [23]:
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, len(labelled_iterator.label_list), True)
model = model.to(DEVICE)

"""
logger = AutoLogger({'precision': lambda gt, pred: (ft.argmax(dim=)).mean()}, tensorboard=True)
for epoch in range(2):
    logger.epoch_start()
    # train_part
    for i in range(2):
        y_true = np.random.randint(0,2, size=(5,))
        y_pred = np.random.randint(0,2, size=(5,))
        logger.log_loss(0, 'train')
        logger.log_metrics(y_true, y_pred, 'train')

    # test part
    for i in range(1):
        y_true = np.random.randint(0,2, size=(5,))
        y_pred = np.random.randint(0,2, size=(5,))
        logger.log_loss(1, 'test')
        logger.log_metrics(y_true, y_pred, 'test')

    logger.epoch_end()
"""
# model = torch.nn.DataParallel(ImModel(models.resnet18, 1000, None), device_ids=[0,1,2,3]).cuda()

optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-3, 
                                          epochs=EPOCHS, 
                                          steps_per_epoch=len(pytorch_dl))


for epoch in tqdm(range(1), leave=True):
    
    epoch_train_loss = 0
    epoch_test_loss = 0
    
    for i, (labelled_data) in enumerate(tqdm(pytorch_dl, total=len(pytorch_dl), leave=False)):
        
        inp, y = labelled_data
        inp = inp.to(DEVICE)
        y = y.reshape(-1).to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inp)

        loss = F.cross_entropy(outputs, y)
        

        loss.backward()
        epoch_train_loss += loss.detach().cpu()

        optimizer.step()
        scheduler.step()
    
    # epoch_train_loss = epoch_train_loss/len(unlabelled_dl)
    # writer.add_scalar('Loss_Epoch', epoch_train_loss, epoch)
    # labelled_loader.reset()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/232 [00:00<?, ?it/s]